# Disability-Adjusted Life Years (DALY) calculation

$YLL$: Years of life lost due to premature mortality

$YLD$: Years of life lost due to disability


## Without discounting or age weighting (GBD post-2012)

$ YLL = \sum_i^{n} \mathbb{1}_{\text{i died}} \times L_i$

where:
* $i$ is the agent's index
* $n$ is the population size
* $\mathbb{1}_{\text{i died}}$ is an indicator function taking the value $1$ if agent $i$ died from Covid-19 and $0$ otherwise
* $L_i$ is the standard life expectancy at agent $i$'s age (average remaining time alive)

$ YLD = \sum_i^{n} \sum_s^{S} DW_s \times L_{i,s}$

where:
* $i$ is the agent's index
* $n$ is the population size
* $S$ is the set of possible health states (does NOT correspond to $S$ $E$ $I$ $R$)
* $s \in S$
* $DW$ is a disability weight associated to state $s$
* $L_{i,s}$ is the duration of state $s$ for agent $i$


## With discounting (GBD 2002)

$\begin{array}{c}
Y L L=\frac{K C e^{r a}}{(r+\beta)^{2}}\left[e^{-(r+\beta)(L+a)}[-(r+\beta)(L+a)-1] -e^{-(r+\beta) a}[-(r+\beta) a-1]\right]+\frac{1-K}{r}\left(1-e^{-r L}\right)
\end{array}$

* $a$ is age of death
* $r$ is social discount rate
* $\beta$ is age weighting constant
* $K$ is age weighting modulation constant
* $C$ is adjustment constant
* $L$ is standard life expectancy at age of death




$Y L D = D W \left\{\frac{K C e^{r a}}{(r+\beta)^{2}}\left[e^{-(r+\beta)(L+a)}[-(r+\beta)(L+a)-1]\right.\right. \left.\left.-e^{-(r+\beta) a}[-(r+\beta) a-1]\right]+\frac{1-K}{r}\left(1-e^{-r L}\right)\right\}$

* $a$ is age of death
* $r$ is social discount rate
* $\beta$ is age weighting constant
* $K$ is age weighting modulation constant
* $C$ is adjustment constant
* $L$ is the duration of the disability
* $DW$ is disability weight 

In [1]:
import pickle
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tkinter

import covid19sim.plotting.plot_dalys as dalys                                        


In [2]:
# STOP! ~2GB of RAM **per intervention** required before proceeding
method_data_dict = {}
interventions = ['post-lockdown-no-tracing', 
                 'bdt1_60',
                 'heuristicv4_60',
                 'heuristicv1_60']

for method in interventions: 
    method_data_dict[method] = dalys.multiple_seeds_get_data(
        method,
        '../qaly_data/life_expectancies/1310011401-eng.csv')

## Tables

In [4]:
dict_dalys_sex_age_total = {}

for key in method_data_dict.keys():
    dict_dalys_sex_age_total[key] = dalys.multiple_seeds_metric(
        dalys.total_dalys_sex_age, method_data_dict[key]['daly_data_dict'])

table_dalys_sex_age_total = pd.concat([pd.concat(dict_dalys_sex_age_total[i], 
                                                 axis=1
                                                )
                                          for i in dict_dalys_sex_age_total.keys()
                                      ],
                                      axis=1,
                                      keys = dict_dalys_sex_age_total.keys()
                                     )

a = table_dalys_sex_age_total.index.to_series().str.rsplit(
    ' - ').str[-1].astype(int).sort_values()
table_dalys_sex_age_total = table_dalys_sex_age_total.reindex(index=a.index)

In [5]:
table_dalys_sex_age_total.rename(index={"100 - 109":'100+'})
table_dalys_sex_age_total.to_csv('../qaly_data/output/tables/dalys_age_sex_total.csv', 
                                 encoding='utf-8',
                                 float_format = '%.6f')

In [11]:
dict_dalys_sex_age_1000 = {}

for key in method_data_dict.keys():
    dict_dalys_sex_age_1000[key] = dalys.multiple_seeds_metric(
        dalys.dalys_per_thousand_sex_age, 
        method_data_dict[key]['daly_data_dict'])

table_dalys_sex_age_1000 = pd.concat([pd.concat(dict_dalys_sex_age_1000[i], 
                                                axis=1) 
                                         for i in dict_dalys_sex_age_1000.keys()
                                     ],
                                     axis=1,
                                     keys = dict_dalys_sex_age_1000.keys())

a = table_dalys_sex_age_1000.index.to_series().str.rsplit(
    ' - ').str[-1].astype(int).sort_values()
table_dalys_sex_age_1000 = table_dalys_sex_age_1000.reindex(index=a.index)
table_dalys_sex_age_1000.rename(index={"100 - 109":'100+'})
table_dalys_sex_age_1000.to_csv('../qaly_data/output/tables/dalys_age_sex_1000.csv', 
                                encoding='utf-8',
                                float_format = '%.6f')

In [12]:
dict_dalys_sex_1000 = {}

for key in method_data_dict.keys():
    dict_dalys_sex_1000[key] = dalys.multiple_seeds_metric(
        dalys.dalys_per_thousand_sex, method_data_dict[key]['daly_data_dict'])

table_dalys_sex_1000 = pd.concat([pd.concat(dict_dalys_sex_1000[i], 
                                            axis=1) 
                                     for i in dict_dalys_sex_1000.keys()
                                 ],
                                 axis=1,
                                 keys = dict_dalys_sex_1000.keys())

table_dalys_sex_1000.to_csv('../qaly_data/output/tables/dalys_sex_1000.csv', 
                            encoding='utf-8', 
                            float_format = '%.6f')

In [13]:
dict_dalys_age_1000 = {}

for key in method_data_dict.keys():
    dict_dalys_age_1000[key] = dalys.multiple_seeds_metric(
        dalys.dalys_per_thousand_age, method_data_dict[key]['daly_data_dict'])

table_dalys_age_1000 = pd.concat([pd.concat(dict_dalys_age_1000[i], axis=1) 
                                     for i in dict_dalys_age_1000.keys()
                                 ],
                                 axis=1,
                                 keys = dict_dalys_age_1000.keys())

a = table_dalys_age_1000.index.to_series().str.rsplit(
    ' - ').str[-1].astype(int).sort_values()
table_dalys_age_1000 = table_dalys_age_1000.reindex(index=a.index)
table_dalys_age_1000.rename(index={"100 - 109":'100+'})
table_dalys_age_1000.to_csv('../qaly_data/output/tables/dalys_age_1000.csv', 
                            encoding='utf-8', 
                            float_format = '%.6f')

In [14]:
dict_dalys_metrics_total = {}

for key in method_data_dict.keys():
    dict_dalys_metrics_total[key] = dalys.multiple_seeds_metric(
        dalys.total_metrics_sex, method_data_dict[key]['daly_data_dict'])

table_dalys_metrics_total = pd.concat([pd.concat(dict_dalys_metrics_total[i], 
                                                 axis=1) 
                                          for i in dict_dalys_metrics_total.keys()
                                      ],
                                      axis=1,
                                      keys = dict_dalys_metrics_total.keys())

a = table_dalys_metrics_total.index.to_series().str.rsplit(
    ' - ').str[-1].astype(int).sort_values()
table_dalys_metrics_total = table_dalys_metrics_total.reindex(index=a.index)
table_dalys_metrics_total.rename(index={"100 - 109":'100+'})
table_dalys_metrics_total.to_csv('../qaly_data/output/tables/dalys_metrics_total.csv', 
                            encoding='utf-8', 
                            float_format = '%.6f')

AttributeError: module 'covid19sim.plotting.plot_dalys' has no attribute 'total_metrics_sex'

In [ ]:
dict_metrics_per_person = {}

for key in method_data_dict.keys():
    dict_metrics_per_person[key] = dalys.multiple_seeds_metric(
        dalys.per_person_metrics_sex_age, method_data_dict[key]['daly_data_dict'])

table_metrics_per_person = pd.concat([pd.concat(dict_metrics_per_person[i], 
                                                 axis=1) 
                                          for i in dict_metrics_per_person.keys()
                                      ],
                                      axis=1,
                                      keys = dict_metrics_per_person.keys())

a = table_metrics_per_person.index.to_series().str.rsplit(
    ' - ').str[-1].astype(int).sort_values()
table_metrics_per_person = table_metrics_per_person.reindex(index=a.index)

table_metrics_per_person = table_metrics_per_person.rename(index={"100 - 109":'100+'})
table_metrics_per_person

table_metrics_per_person.to_csv('../qaly_data/output/tables/dalys_metrics_per_person.csv', 
                            encoding='utf-8', 
                            float_format = '%.6f')

## plots

In [20]:
plt.figure(figsize=(15,10))

for method in dict_dalys_age_1000.keys():
    mean = dict_dalys_age_1000[method]['mean'].values
    stderr = dict_dalys_age_1000[method]['std_err'].values
    plt.plot(mean, label = method)
    plt.fill_between(list(range(len(mean))), 
                     mean-stderr, 
                     mean+stderr, 
                     alpha = 0.2)
    
plt.xticks([i for i in range(0,11)], 
           [str(i) + ' to ' + str(i+9) + ' years' for i in range(0,100,10)]+ ['100+ years'],
           rotation=60, 
           ha='right', 
           fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(fontsize = 16)
plt.title('Dalys per thousand, stratified by age group', fontsize=24)
plt.tight_layout()
plt.savefig('../qaly_data/output/graphs/dalys_1000_age_10.png')


In [16]:
dict_dalys_total = {}
dict_work_1000 = {}

for key in method_data_dict.keys():
    dict_dalys_total[key] = dalys.multiple_seeds_metric(
        dalys.total_dalys, method_data_dict[key]['daly_data_dict'])
    dict_work_1000[key] = dalys.multiple_seeds_metric(
        dalys.lost_work_hours_total, method_data_dict[key]['tracker_dict'])
    

In [1]:
plt.figure(figsize=(15,10))

for method in method_data_dict.keys():
    plt.scatter(dict_work_1000[method]['mean'], 
                dict_dalys_total[method]['mean'],
                label = method)
    plt.errorbar(dict_work_1000[method]['mean'], 
                dict_dalys_total[method]['mean'],
                xerr = dict_work_1000[method]['std_err'],
                yerr = dict_dalys_total[method]['std_err'])

plt.ylim(bottom = 0)
plt.xlim(left = 0)
plt.title('Tradeoff between DALYs and foregone work hours, 60% adoption rate', fontsize=24)
plt.legend()
plt.tight_layout()
plt.savefig('../qaly_data/output/graphs/pareto_comparison.png')

NameError: name 'plt' is not defined

## Scalar calculations

In [9]:
temp = pd.read_csv('../qaly_data/output/tables/dalys_age_sex_total.csv', header = 3)

In [10]:
temp

,Unnamed: 0,male,female,male.1,female.1,male.2,female.2,male.3,female.3,male.4,...,male.7,female.7,male.8,female.8,male.9,female.9,male.10,female.10,male.11,female.11
0,Age,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0 - 9,0.103998,0.102545,0.013018,0.010263,0.004117,0.003245,0.092359,0.095712,0.015848,...,0.006754,0.014464,0.002136,0.004574,0.037894,0.038802,0.012442,0.014273,0.003935,0.004514
2,10 - 19,0.156432,0.142366,0.017302,0.013243,0.005471,0.004188,0.153500,0.139096,0.013754,...,0.009471,0.012123,0.002995,0.003834,0.063695,0.058538,0.008417,0.014488,0.002662,0.004582
3,20 - 29,0.232804,0.234353,0.016873,0.018718,0.005336,0.005919,0.229225,0.232946,0.015352,...,0.019739,0.017087,0.006242,0.005403,0.087795,0.087184,0.015336,0.019217,0.004850,0.006077
4,30 - 39,0.270965,5.186875,0.017970,15.498902,0.005683,4.901183,0.269141,5.173374,0.022364,...,0.024310,15.495490,0.007687,4.900104,0.128418,5.043610,0.024736,15.504293,0.007822,4.902888
5,40 - 49,7.779844,0.296646,15.748550,0.017143,4.980129,0.005421,7.771556,0.294761,15.747177,...,15.750173,0.016603,4.980642,0.005250,7.599447,0.129322,15.758315,0.018619,4.983217,0.005888
6,50 - 59,3.490429,13.311689,10.036848,16.843053,3.173930,5.326241,3.482845,13.303804,10.037601,...,0.020503,16.255179,0.006484,5.140339,0.125751,3.676573,0.016832,11.218461,0.005323,3.547589
7,60 - 69,6.767936,14.219136,14.577089,19.777809,4.609680,6.254292,6.336828,14.469078,9.996316,...,0.010079,9.446167,0.003187,2.987140,0.049658,2.252102,0.014128,6.961867,0.004468,2.201536
8,70 - 79,7.153511,11.474118,13.447865,11.318395,4.252588,3.579191,5.807342,7.977530,9.834385,...,4.056681,11.813456,1.282835,3.735743,4.240876,7.838135,9.282365,10.700279,2.935342,3.383725
9,80 - 89,3.792949,8.037235,4.117256,5.567579,1.301991,1.760623,3.093248,7.665152,4.098248,...,2.883637,5.385157,0.911886,1.702936,2.033001,3.105780,3.256550,4.332642,1.029811,1.370102


## Turn Back: debugging + next steps

In [4]:
temp = dalys.multiple_seeds_metric(dalys.per_person_metrics_sex_age, a['daly_data_dict'])
temp['mean']
temp2 = temp['mean'].index.to_series().str.rsplit(
    ' - ').str[-1].astype(int).sort_values()
temp['mean'] = temp['mean'].reindex(index=temp2.index)
temp['mean'].rename(index={"100 - 109":'100+'})
temp['mean']

sex                 male                                         female  \
metric    YLL per person YLD per person DALYs per person YLL per person   
Age                                                                       
0 - 9           0.000000       0.000716         0.000716       0.000000   
10 - 19         0.000000       0.001169         0.001169       0.000000   
20 - 29         0.000000       0.001149         0.001149       0.000000   
30 - 39         0.000000       0.001341         0.001341       0.022791   
40 - 49         0.040984       0.001788         0.042772       0.000000   
50 - 59         0.016010       0.001852         0.017862       0.071544   
60 - 69         0.043184       0.001236         0.044420       0.079849   
70 - 79         0.125665       0.001911         0.127576       0.182534   
80 - 89         0.087586       0.001545         0.089131       0.234459   
90 - 99         0.124995       0.001927         0.126921       0.166989   
100 - 109       0.101608       0.002229         0.103837       0.090156   

sex                                        
metric    YLD per person DALYs per person  
Age                                        
0 - 9           0.000737         0.000737  
10 - 19         0.001110         0.001110  
20 - 29         0.001186         0.001186  
30 - 39         0.001343         0.024134  
40 - 49         0.001706         0.001706  
50 - 59         0.001893         0.073437  
60 - 69         0.001257         0.081107  
70 - 79         0.002011         0.184544  
80 - 89         0.001644         0.236103  
90 - 99         0.001869         0.168857  
100 - 109       0.002120         0.092277

In [117]:
# df = pd.read_csv('qaly_data/gbd/IHME_GBD_2017_DISABILITY_WEIGHTS_Y2018M11D08.csv',
#                 header = 1)
# df[df['Sequela'] == 'Chronic obstructive pulmonary disease']

# life_expectancies_preexisting = pd.read_csv('deaths_yll_IHME-GBD_2017_DATA-644592d9-1.csv')
# conditions_dict ={'COPD' : 'Chronic obstructive pulmonary disease',
# #                  'allergies', #not in gbd
#                  'asthma': 'Asthma',
# #                  'cancer', #not in gbd
#                  'diabetes':'Diabetes mellitus',
#                  'heart_disease':'Ischemic heart disease',
#                  'immuno-suppressed':'Endocrine, metabolic, blood, and immune disorders',
#                  'lung_disease':'Interstitial lung disease and pulmonary sarcoidosis',
# #                  'pregnant', #not in gbd
# #                  'smoker', #not in gbd
#                  'stroke': 'Stroke'}
# le_preexisting_dfs = {}
# for key in conditions_dict.keys():
#     le_preexisting_dfs[key] = life_expectancies_preexisting[life_expectancies_preexisting.cause == conditions_dict[key]]

# le_preexisting_dfs['COPD'][le_preexisting_dfs['COPD'].sex == 'Male'].groupby('age').agg({'YLL'})

# test_df = le_preexisting_dfs['COPD'][(le_preexisting_dfs['COPD'].sex == 'Male') 
#                            & (le_preexisting_dfs['COPD'].age == '1 to 4')]


# val = test_df[test_df.measure == 'YLLs (Years of Life Lost)']['val'].item() / \
#       test_df[test_df.measure == 'Deaths']['val'].item()
# val

# set(life_expectancies.measure.values)

# life_expectancies[life_expectancies['Sex'] == 'other']

In [ ]:
# symptom_severity = {}
# for day in tracker_data['human_monitor'].keys():
#     symptom_severity[day] = {}
#     for person in range(tracker_data['n_humans']):
#         symptom_severity[day][tracker_data['human_monitor'][day][person]['name']] \
#         = tracker_data['human_monitor'][day][person]['test_result']


# tracker_data['human_monitor'][datetime.date(2020,3,28)][0]

Non-parallelized functions

In [ ]:
# def get_age(human_name, 
#             demographics_data = demographics):
#     df_demographics = pd.DataFrame(demographics_data).set_index('name')
#     #returns integer age
#     return df_demographics['age'][human_name]

# def get_sex(human_name, 
#             demographics_data = demographics):
#     df_demographics = pd.DataFrame(demographics_data).set_index('name')
#     # returns 'male', 'female', or 'other'
#     return df_demographics['sex'][human_name]

# def get_preexisting(human_name, 
#                     demographics_data = demographics):
#     df_demographics = pd.DataFrame(demographics_data).set_index('name')
#     # returns a list of preexisting conditions
#     return df_demographics['preexisting_conditions'][human_name]

# def has_symptoms_and_infection(human_name, 
#                    human_monitor_data = human_monitor_data):
#     has_symptoms = {}
#     for day in human_monitor_data.keys():
#         df_human_monitor_data = pd.DataFrame(human_monitor_data[day]).set_index('name')
#         n_symptoms = df_human_monitor_data['n_symptoms'][human_name]
#         has_infection = df_human_monitor_data['infection_timestamp'][human_name] is not None
#         has_symptoms[day] = n_symptoms>0 and has_infection
#     # returns a timeseries of boolean values    
#     return has_symptoms

    
# def get_hospitalization(human_name, 
#                         human_monitor_data = human_monitor_data):
#     hospitalization_status = {}
#     for day in human_monitor_data.keys():
#         df_human_monitor_data = pd.DataFrame(human_monitor_data[day]).set_index('name')
#         is_in_hospital = df_human_monitor_data['is_in_hospital'][human_name]
#         hospitalization_status[day] = is_in_hospital
#     # returns a timeseries of boolean values    
#     return hospitalization_status

# def get_critical(human_name, 
#                  human_monitor_data = human_monitor_data):
#     ICU_status = {}
#     for day in human_monitor_data.keys():
#         df_human_monitor_data = pd.DataFrame(human_monitor_data[day]).set_index('name')
#         is_in_ICU = df_human_monitor_data['in_in_ICU'][human_name]
#         ICU_status[day] = is_in_ICU
#     # returns a timeseries of boolean values    
#     return ICU_status

# def get_died(human_name,
#              human_monitor_data = human_monitor_data):
#     death_status = {}
#     for day in human_monitor_data.keys():
#         df_human_monitor_data = pd.DataFrame(human_monitor_data[day]).set_index('name')
#         died = df_human_monitor_data['dead'][human_name]
#         death_status[day] = died
#     # returns a timeseries of boolean values    
#     return sum(death_status.values())


# def get_life_expectancy(human_name, 
#                         demographics_data = demographics, 
#                         life_expectancies = life_expectancies):
#     """
#         If age = 25 and expected age of death is 100, then l.e. is 75.
#         Standard life expectancy will depend on current age, sex and preexisting.
#         Specfically, it does NOT depend on hospitalization status associated to Covid.
#     """
#     # TODO: include preexisting conditions in life expectancy
#     age = get_age(human_name, demographics_data)
#     sex = get_sex(human_name, demographics_data)
#     preexisting = get_preexisting(human_name, demographics_data)
#     current_LE = life_expectancies[sex][str(age)+' years']
    
#     return current_LE

# def get_duration_disability(human_name,
#                             human_monitor_data,
#                             method = 'infection_and_symptomatic'):
#     """
#         There are multiple possibilities for duration disability.
#         Duration of disability can depend on simply being infected
#                                              hospitalization/critical care status
#                                              the presence of symptoms
#     """
#     if method == 'infection_and_symptomatic':
#         return sum(has_symptoms_and_infection(human_name,
#                                               human_monitor_data = human_monitor_data
#                                               ).values())
    
#     elif method == 'hospitalization':
#         time_in_hospital = get_hospitalization(human_name,
#                                               human_monitor_data = human_monitor_data
#                                                )
#         time_in_ICU = get_critical(human_name, 
#                                    human_monitor_data = human_monitor_data)
#         return {'time_in_hospital': sum(time_in_hospital.values()),
#                 'time_in_ICU': sum(time_in_ICU.values())}
    
#     elif method == 'symptoms':
#         #### TODO
#         pass 